In [ ]:
#!gdown 1KZQErVGYSlcv8ukwaXUkelBYdAsKIggn
#!unzip dataset_raw_080525.zip
#!rm dataset_raw_080525.zip

In [ ]:
import torch
import matplotlib.pyplot as plt
from gaussian_net import ImagePairDataset

device = "cuda"
orig_imgs = torch.load('dataset_raw/long_exp.pt')
filter_imgs = torch.load('dataset_raw/filter_long_exp.pt')


dataset = ImagePairDataset(orig_imgs.permute(0, 3, 1, 2), filter_imgs.permute(0, 3, 1, 2))
dataloader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

/tmp/ipykernel_1486753/1133984056.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  orig_imgs = torch.load('dataset_raw/long_exp.pt')
/tmp/ipykernel_1486753/1133984056.py:

In [2]:
from gaussian_net import GaussianNet, set_seed

set_seed(42)
net = GaussianNet(3).to(device)

In [ ]:
from torch import nn, optim

criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=2000)

losses = []
num_epochs = 10

for epoch in range(num_epochs):
    for original_img, filter_img in dataloader:
        original_img = original_img.to(device)
        filter_img = filter_img.to(device)

        optimizer.zero_grad()
        blurred_img = net(original_img)
        loss = criterion(blurred_img, filter_img)
        loss.backward()
        optimizer.step()
        scheduler.step()
        losses.append(loss.item())

    print(f"Epoch {epoch}, Loss: {loss.item()}")

print("Finished Training")

Epoch 0, Loss: 0.02413240820169449
Epoch 1, Loss: 0.03141651675105095
Epoch 2, Loss: 0.045433852821588516
Epoch 3, Loss: 0.049524448812007904
Epoch 4, Loss: 0.08734865486621857
Epoch 5, Loss: 0.0289719607681036
Epoch 6, Loss: 0.062271859496831894


In [ ]:
plt.plot(losses)

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 3, 1)
plt.imshow(original_img.squeeze().permute(1, 2, 0).cpu().numpy())
plt.title("Original Image")
plt.axis("off")


plt.subplot(1, 3, 2)
plt.imshow(filter_img.squeeze().permute(1, 2, 0).cpu().numpy())
plt.title("Filtered Image")
plt.axis("off")

plt.subplot(1, 3, 3)
plt.imshow(blurred_img.squeeze().detach().permute(1, 2, 0).cpu().numpy())
plt.title("Blurred Image")
plt.axis("off")


plt.tight_layout()
plt.show()